In [9]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model

# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
#weights_path = '../model/vgg16_weights.h5'
top_model_weights_path = '../model/bottleneck_fc_model.h5'
#top_model_weights_path = '../model/fc_model.h5'
img_width, img_height = 150, 150

#small
#train_data_dir = '../data/Pandora18K_small_train_val_test_split/train'
#validation_data_dir = '../data/Pandora18K_small_train_val_test_split/val'
#nb_train_samples = 1462
#nb_validation_samples = 167
#total_num_classes = 18

#verysmall
train_data_dir = '../data/verysmall/train'
validation_data_dir = '../data/verysmall/val'
nb_train_samples = 144 #157
nb_validation_samples = 16 #18
total_num_classes = 2

epochs = 2
batch_size = 16

In [10]:

# build the VGG16 network
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
"""
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
#top_model.add(Dropout(0.5))
top_model.add(Dense(total_num_classes, activation='sigmoid'))
"""


top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dropout(0.5))
top_model.add(Dense(1024, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(total_num_classes, activation='softmax'))



# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
# model.add(top_model)

model = Model(input= base_model.input, output= top_model(base_model.output))

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)


Model loaded.
Found 144 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Epoch 1/5
9/9 [==============================] - 5s - loss: 7.2668 - acc: 0.5486 - val_loss: 7.0517 - val_acc: 0.5625
Epoch 2/5
9/9 [==============================] - 4s - loss: 7.2755 - acc: 0.5486 - val_loss: 7.0517 - val_acc: 0.5625
Epoch 3/5
9/9 [==============================] - 4s - loss: 7.2755 - acc: 0.5486 - val_loss: 7.0517 - val_acc: 0.5625
Epoch 4/5
9/9 [==============================] - 4s - loss: 7.2755 - acc: 0.5486 - val_loss: 7.0517 - val_acc: 0.5625
Epoch 5/5
9/9 [==============================] - 4s - loss: 7.2755 - acc: 0.5486 - val_loss: 7.0517 - val_acc: 0.5625


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:33: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:73: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=9, epochs=5, validation_steps=16)`
